<a href="https://colab.research.google.com/github/HiimanshuSherawat/DLCVNLP/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 6s - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160 - 6s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.7955 - accuracy: 0.7361 - val_loss: 0.7130 - val_accuracy: 0.7656 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.5553 - accuracy: 0.8156 - val_loss: 0.5350 - val_accuracy: 0.8238 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.5339 - accuracy: 0.8223 - val_loss: 0.5156 - val_accuracy: 0.8302 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.5173 - accuracy: 0.8272 - val_loss: 0.5079 - val_accuracy: 0.8284 - 5s/epoch - 3ms/step
Epoch 9/

In [7]:
model.save("full_mnist_model.h5")

***Transfer*** **Learning**

In [8]:
load_full_mnist_model = tf.keras.models.load_model("full_mnist_model.h5")

In [9]:
load_full_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [10]:
np.where(y_train % 2 == 0, 1 , 0)

array([1, 1, 0, ..., 0, 1, 0])

In [11]:
y_train[0], y_train[-1] 

(4, 5)

In [12]:
def update_even_odd_labels(labels):
    for idx, label in enumerate(labels):
        labels[idx] = np.where(label % 2 == 0, 1 , 0)
    return labels

In [13]:
y_train_bin, y_valid_bin, y_test_bin = update_even_odd_labels([y_train, y_valid, y_test])

In [14]:
# Check trainable or not:

for layer in load_full_mnist_model.layers:
    print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


In [15]:
# make lower layer UN-trainable except the last layer:

for layer in load_full_mnist_model.layers[:-1]:
    layer.trainable = False
    print(f"Now {layer.name}: {layer.trainable}")

Now flatten: False
Now dense: False
Now leaky_re_lu: False
Now dense_1: False
Now leaky_re_lu_1: False


In [16]:
new_model = tf.keras.models.Sequential(load_full_mnist_model.layers[:-1])
new_model.add(
    tf.keras.layers.Dense(1, activation="sigmoid")
)

In [17]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 265,701
Trainable params: 101
Non-traina

In [18]:
new_model.get_config()

{'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 28, 28),
    'dtype': 'float32',
    'name': 'flatten_input',
    'ragged': False,
    'sparse': False}},
  {'class_name': 'Flatten',
   'config': {'batch_input_shape': (None, 28, 28),
    'data_format': 'channels_last',
    'dtype': 'float32',
    'name': 'flatten',
    'trainable': False}},
  {'class_name': 'Dense',
   'config': {'activation': 'linear',
    'activity_regularizer': None,
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'dtype': 'float32',
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'HeNormal', 'config': {'seed': None}},
    'kernel_regularizer': None,
    'name': 'dense',
    'trainable': False,
    'units': 300,
    'use_bias': True}},
  {'class_name': 'LeakyReLU',
   'config': {'alpha': 0.30000001192092896,
    'dtype': 'float32',
    'name': 'leaky_re_lu',
    'trainable': False}}

In [19]:
# Check trainable or not:

for layer in new_model.layers:
    print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_3: True


In [20]:
new_model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [21]:
new_history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 4s - loss: 0.2867 - accuracy: 0.9086 - val_loss: 0.2017 - val_accuracy: 0.9426 - 4s/epoch - 2ms/step
Epoch 2/10
1719/1719 - 3s - loss: 0.1842 - accuracy: 0.9435 - val_loss: 0.1707 - val_accuracy: 0.9466 - 3s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 3s - loss: 0.1651 - accuracy: 0.9471 - val_loss: 0.1583 - val_accuracy: 0.9474 - 3s/epoch - 2ms/step
Epoch 4/10
1719/1719 - 3s - loss: 0.1558 - accuracy: 0.9488 - val_loss: 0.1514 - val_accuracy: 0.9490 - 3s/epoch - 2ms/step
Epoch 5/10
1719/1719 - 3s - loss: 0.1500 - accuracy: 0.9498 - val_loss: 0.1466 - val_accuracy: 0.9512 - 3s/epoch - 2ms/step
Epoch 6/10
1719/1719 - 3s - loss: 0.1459 - accuracy: 0.9508 - val_loss: 0.1432 - val_accuracy: 0.9526 - 3s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 3s - loss: 0.1427 - accuracy: 0.9517 - val_loss: 0.1405 - val_accuracy: 0.9542 - 3s/epoch - 2ms/step
Epoch 8/10
1719/1719 - 3s - loss: 0.1403 - accuracy: 0.9522 - val_loss: 0.1384 - val_accuracy: 0.9542 - 3s/epoch - 2ms/step
Epoch 9/

In [22]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.1442 - accuracy: 0.9520


[0.14422181248664856, 0.9520000219345093]

In [23]:
X_new = X_test[:3]
y_test[:3], y_test_bin[:3]

(array([9, 2, 1], dtype=uint8), array([0, 1, 0]))

In [24]:
new_model.predict(X_new)

array([[0.00803134],
       [0.99993324],
       [0.00607854]], dtype=float32)